# Notebook for filter, anlgn and build phylogenetic tree for TBEV

## Load the fasta file and filter the sequences less than 10k base

In [1]:
WKDIR = '/home/jovyan/data/SJ-data/'

In [2]:
import csv
import Bio.Seq
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML

In [3]:
def readSeq(WKDIR,filename,lengthRequire):
    seqs = {}
    with open(WKDIR+filename,'r') as f:
        for i in SeqIO.parse(f,'fasta'):
            if len(i.seq) > lengthRequire:
                seqs[i.description.split(' ')[0].split('.')[0]]=str(i.seq)
    return seqs

In [4]:
def separateMeta(WKDIR,filename,filteredSegment):
    metadataDict = {}
    if filteredSegment == 0:
        with open(WKDIR+filename,'r') as f:
            for i in SeqIO.parse(f,'fasta'):
                if '-' in i.description.split('|')[2]:
                    metadataDict[i.description.split('|')[0].split('.')[0]] = i.description.split('|')[1] + '_' + i.description.split('|')[2].split('-')[0]
                else:
                    metadataDict[i.description.split('|')[0].split('.')[0]] = i.description.split('|')[1] + '_' + i.description.split('|')[2]
    else:
        with open(WKDIR+filename,'r') as f:
            for i in SeqIO.parse(f,'fasta'):
                if i.description.split('|')[3] == str(filteredSegment):
                    if '-' in i.description.split('|')[2]:
                        metadataDict[i.description.split('|')[0].split('.')[0]] = i.description.split('|')[1] + '_' + i.description.split('|')[2].split('-')[0]
                    else:
                        metadataDict[i.description.split('|')[0].split('.')[0]] = i.description.split('|')[1] + '_' + i.description.split('|')[2]
    return metadataDict

In [5]:
def writeSeq(WKDIR,seqDict,metadataDict,filename):
    for i in metadataDict:
        if 'NC_'not in i:
            standartLen = len(i.split('_'))
            
    with open (WKDIR+filename,'w') as f:
        for i in seqDict:
            if 'NC_' not in i:
                if '_' in i:
                    metaID = i.split('_')[0]
                    f.write('>'+i+'\n'+seqDict[i].lower()+'\n')
                else:
                    f.write('>'+i+'_'+metadataDict[i]+'\n'+seqDict[i].lower()+'\n')
            else:
                if len(i.split('_')) > standartLen:
                    metaID = i.split('_')[0]+'_'+i.split('_')[1]
                    f.write('>'+i+'\n'+seqDict[i].lower()+'\n')
                else:
                    f.write('>'+i+'_'+metadataDict[i]+'\n'+seqDict[i].lower()+'\n')

In [6]:
def writeSeqTrimHyphen(WKDIR,seqDict,metadataDict,filename):
    for i in metadataDict:
        if 'NC_'not in i:
            standartLen = len(i.split('_'))
            
    with open (WKDIR+filename,'w') as f:
        for i in seqDict:
            if 'NC_' not in i:
                if '_' in i:
                    metaID = i.split('_')[0]
                    f.write('>'+i+'\n'+seqDict[i].lower().replace('-','')+'\n')
                else:
                    f.write('>'+i+'_'+metadataDict[i]+'\n'+seqDict[i].lower().replace('-','')+'\n')
            else:
                if len(i.split('_')) > standartLen:
                    metaID = i.split('_')[0]+'_'+i.split('_')[1]
                    f.write('>'+i+'\n'+seqDict[i].lower().replace('-','')+'\n')
                else:
                    f.write('>'+i+'_'+metadataDict[i]+'\n'+seqDict[i].lower().replace('-','')+'\n')

In [7]:
def writeSeq_meta(WKDIR,seqDict,metadataDict,filename):
    for i in metadataDict:
        if 'NC_'not in i:
            standartLen = len(i.split('_'))
            
    with open (WKDIR+filename,'w') as f:
        for i in seqDict:
            if 'NC_' not in i and i in metadataDict:
                if '_' in i:
                    metaID = i.split('_')[0]
                    f.write('>'+i+'\n'+seqDict[i].lower()+'\n')
                else:
                    f.write('>'+i+'_'+metadataDict[i]+'\n'+seqDict[i].lower()+'\n')
            elif i in metadataDict:
                if len(i.split('_')) > standartLen:
                    metaID = i.split('_')[0]+'_'+i.split('_')[1]
                    f.write('>'+i+'\n'+seqDict[i].lower()+'\n')
                else:
                    f.write('>'+i+'_'+metadataDict[i]+'\n'+seqDict[i].lower()+'\n')

In [8]:
def selectRegion(seqDict,startCodon,endCodon):
    newDict = {}
    for i in seqDict:
        newDict[i] = seqDict[i][startCodon-1:endCodon]
    return newDict

In [9]:
def fliterHyphen(seqDict,numberOfHyphen):
    newDict = {}
    for i in seqDict:
        count = 0
        for char in seqDict[i]:
            if char == '-':
                count += 1
        if count <= numberOfHyphen:
            newDict[i] = seqDict[i]
    return newDict
        

In [10]:
def extractTrimList(WKDIR,trimfile,originalfile,outputfile):
    IDlist = []
    seqs = {}
    with open(WKDIR+trimfile,'r') as f:
        for i in SeqIO.parse(f,'fasta'):
            IDlist.append(i.description)
    with open(WKDIR+originalfile,'r') as f:
        for i in SeqIO.parse(f,'fasta'):
            seqs[i.description.split(' ')[0].split('.')[0]]=str(i.seq)
    with open(WKDIR+outputfile,'w') as f:
        for i in IDlist:
            f.write('>'+i+'\n'+seqs[i]+'\n')       

In [11]:
def findNCandWritetoFile(WKDIR,RefID,inputfile,virusName):
    seqs = {}
    with open(WKDIR+inputfile,'r') as f:
        for i in SeqIO.parse(f,'fasta'):
            if RefID in i.description:
                with open(WKDIR+virusName+'-Ref-Region.fasta','w') as f:
                    f.write('>'+i.description+'\n'+str(i.seq)+'\n')
    

In [12]:
def combineSeq(seqDict1,seqDict2):
    combinedSeq = {}
    for i in seqDict1:
        combinedSeq[i] = seqDict1[i]
    for i in seqDict2:
        combinedSeq[i] = seqDict2[i]
    return combinedSeq

In [13]:
def dedupNC(WKDIR,inputfile,outputfile):
    newseqs = {}
    with open(WKDIR+inputfile,'r') as f:
        for i in SeqIO.parse(f,'fasta'):
            newseqs[i.description] = str(i.seq)
    with open(WKDIR+outputfile,'w') as f:
        for i in newseqs:
            f.write('>'+i+'\n'+newseqs[i]+'\n')
    

In [14]:
def translate(WKDIR,seqDict,protein_name,start_coden,end_coden):
    with open(WKDIR+protein_name+'.fasta','w') as f:
        for i in seqDict:
            seq = Bio.Seq.translate(seqDict[i][start_coden-1:end_coden].replace('-','n'))
            if 'NC' not in i:
                f.write('>'+i+'\n'+seq+'\n')
            else:
                f.write('>'+i+'\n'+seq+'\n')

In [53]:
def reverseComplement(WKDIR,inputfile,outputfile):
    newseqs = {}
    with open(WKDIR+inputfile,'r') as f:
        for i in SeqIO.parse(f,'fasta'):
            newseqs[i.description] = str(i.seq.reverse_complement())
    with open(WKDIR+outputfile,'w') as f:
        for i in newseqs:
            f.write('>'+i+'\n'+newseqs[i]+'\n')

In [87]:
seqs = readSeq(WKDIR,'Astrovirus.fasta',0)

In [88]:
metadataDict = separateMeta(WKDIR,'Astrovirus.fasta',0)

In [89]:
len(seqs)
len(metadataDict)

153

In [90]:
writeSeq_meta(WKDIR,seqs,metadataDict,'CovertID_Astrovirus.fasta')

In [91]:
!mafft --thread 60 --auto --keeplength --addfragments /home/jovyan/data/SJ-data/CovertID_Astrovirus.fasta /home/jovyan/data/SJ-data/Astrovirus-Ref.fasta > /home/jovyan/data/SJ-data/aligned_Astrovirus.fasta | tail -n 10

nadd =  153
npair =  153
nseq =  154
nlen =  6924
use ktuples, size=6!
nadd = 153
ppenalty_ex = -10
nthread = 60
blosum 62 / kimura 200
sueff_global = 0.100000
norg = 1
njobc = 2
generating a scoring matrix for nucleotide (dist=200) ... done


Making a distance matrix ..

There are 20 ambiguous characters
    1 / 1 (thread    0)
done.

fTEP 152 / 153 (thread 52)                    ff

Combining ..
   done.                      

   done.                      

addsingle (nuc) Version 7.520
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
60 thread(s)


To keep the alignment length, 7908 letters were DELETED.
To know the positions of deleted letters, rerun the same command with the --mapout option.

Strategy:
 FFT-NS-fragment (Not tested.)
 ?

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It te

In [92]:
alignedSeq = readSeq(WKDIR,'aligned_Astrovirus.fasta',0)

In [93]:
selectSeq = selectRegion(alignedSeq,4555,5005)

In [94]:
#writeSeq(WKDIR,seqDict,metadataDict,filename)
writeSeq(WKDIR,selectSeq,metadataDict,'Astrovirus_4555_5005.fasta')

In [95]:
filteredSeq = fliterHyphen(selectSeq,0)

In [96]:
len(filteredSeq)

146

In [97]:
#def writeSeq(WKDIR,seqDict,metadataDict,filename):
writeSeqTrimHyphen(WKDIR,filteredSeq,metadataDict,'filtered_for_trim_Astrovirus.fasta')

In [98]:
!cd-hit -c 0.98 -i filtered_for_trim_Astrovirus.fasta -o trim_Astrovirus.fasta | tail -n 10

comparing sequences from          0  to        146

      146  finished         42  clusters

Approximated maximum memory consumption: 81M
writing new database
writing clustering information
program completed !

Total CPU time 0.09


In [99]:
# !iqtree -s trim_Rotavirus_A.fasta -m MFP -bb 1000 -T AUTO -redo -o 'NC_011503'

In [100]:
findNCandWritetoFile(WKDIR,'NC_001943','trim_Astrovirus.fasta','Astrovirus')

In [101]:
reverseComplement(WKDIR,'Astrovirus_exp.fasta','RC_Astrovirus_exp.fasta')

In [102]:
refSeqDict = readSeq(WKDIR,'trim_Astrovirus.fasta',0)
expSeqDict = readSeq(WKDIR,'RC_Astrovirus_exp.fasta',0)

In [103]:
unify = combineSeq(refSeqDict,expSeqDict)

In [104]:
writeSeq(WKDIR,unify,metadataDict,'Unified_trim_Astrovirus_with_exp.fasta')

In [105]:
!mafft --thread 36 --auto --keeplength --addfragments /home/jovyan/data/SJ-data/Unified_trim_Astrovirus_with_exp.fasta /home/jovyan/data/SJ-data/Astrovirus-Ref-Region.fasta > /home/jovyan/data/SJ-data/aligned_Astrovirus_exp.fasta | tail -n 10

nadd =  57
npair =  57
nseq =  58
nlen =  465
use multipair, weighti=2.7!
nadd = 57
generating a scoring matrix for nucleotide (dist=200) ... done
dndpre (nuc) Version 7.520
alg=X, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
36 thread(s)

generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
    0 / 1 (by thread   0) 

##### writing hat3
pairlocalalign (nuc) Version 7.520
alg=Y, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
36 thread(s)

nadd = 57
ppenalty_ex = -10
nthread = 36
blosum 62 / kimura 200
sueff_global = 0.100000
norg = 1
njobc = 2
Loading 'hat3' ... 
done.
generating a scoring matrix for nucleotide (dist=200) ... done
Loading 'hat2n' (aligned sequences - new sequences) ... done.
Loading 'hat2i' (aligned sequences) ... done.
cTEP 56 / 57 (thread 10)                    c

Combining ..
   done.                      

   done.                      

addsingle (nuc) Version 7.520
alg=A, model=DNA200 (2), 1.53 (

In [106]:
dedupNC(WKDIR,'aligned_Astrovirus_exp.fasta','aligned_Astrovirus_exp.fasta')

In [107]:
!iqtree -s aligned_Astrovirus_exp.fasta -m MFP -bb 1000 -T 4 -redo -o 'NC_001943' | tail -n 10

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
  IQ-TREE report:                aligned_Astrovirus_exp.fasta.iqtree
  Maximum-likelihood tree:       aligned_Astrovirus_exp.fasta.treefile
  Likelihood distances:          aligned_Astrovirus_exp.fasta.mldist

Ultrafast bootstrap approximation results written to:
  Split support values:          aligned_Astrovirus_exp.fasta.splits.nex
  Consensus tree:                aligned_Astrovirus_exp.fasta.contree
  Screen log file:               aligned_Astrovirus_exp.fasta.log

Date and Time: Thu Jul 20 09:26:25 2023


In [108]:
seqForTranslate = readSeq(WKDIR,'aligned_Astrovirus_exp.fasta',0)

In [109]:
translate(WKDIR,seqForTranslate,'Astrovirus-AA',2,999)